In [4]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY"])


In [5]:

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
generation_config = {
"temperature": 0,
"top_p": 0.95,
"top_k": 40,
"max_output_tokens": 8192,
"response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
model_name="gemini-1.5-pro",
generation_config=generation_config,
)

def call_gemini_with_image(image_path, prompt, mime_type="image/jpeg"):
  

  # Upload the image file
  file = upload_to_gemini(image_path, mime_type=mime_type)
  
  # Start a chat session with the image
  chat_session = model.start_chat(
    history=[
      {
        "role": "user",
        "parts": [
          file,
        ],
      },
    ]
  )

  # Send the prompt and get the response
  response = chat_session.send_message(prompt)
  
  return response.text



In [9]:
prompt ="""
Analyze this street view image and provide a comprehensive safety risk assessment based on the following parameters:

1. Road Infrastructure:
   - Road condition (potholes, cracks, uneven surfaces)
   - Lane markings and visibility
   - Traffic signals and signs (presence, visibility, condition)
   - Intersections and junctions (design, visibility)
   - Pedestrian infrastructure (sidewalks, crossings, barriers)

2. Environmental Factors:
   - Lighting conditions (adequate for visibility)
   - Weather impact on road conditions
   - Visibility issues (fog, glare, obstructions)
   - Natural hazards (flooding areas, landslide prone)
   - Time of day considerations

3. Human Factors:
   - Pedestrian density and behavior
   - School zones or other special areas
   - Commercial activity level
   - Evidence of traffic rule violations
   - Population density indicators

4. Infrastructure Quality:
   - Road maintenance level
   - Construction zones or temporary hazards
   - Utility infrastructure condition (poles, wires)
   - Drainage systems
   - Barriers and guardrails

5. Location-Specific Risks:
   - Evidence of previous accidents
   - Blind spots or visual obstructions
   - Unusual road geometry (sharp turns, steep grades)
   - Unauthorized structures or encroachments
   - Animal crossing areas

Analyze this street view image and provide a safety risk assessment in the following JSON format:
{
"score": [a number from 1-100 where 1 is extremely dangerous and 100 is very safe],
"explanation": [a brief summary of the key safety concerns or positive safety features observed in the image]
}

Focus only on what is visible in the image.
"""

In [11]:
response_text = call_gemini_with_image("pic.png", prompt)
print(response_text)

Uploaded file 'pic.png' as: https://generativelanguage.googleapis.com/v1beta/files/hgcxcwkhh3i4
```json
{
  "score": 65,
  "explanation": "The road appears to be in reasonably good condition with visible lane markings. However, the lack of dedicated pedestrian infrastructure like sidewalks or crosswalks poses a risk to pedestrians sharing the road with motorized vehicles.  The presence of parked vehicles and informal commercial activity along the roadside could further obstruct visibility and increase the risk of accidents. While the lighting conditions seem adequate for daytime, the image doesn't provide information about nighttime visibility.  No obvious traffic signals are visible, and the general environment suggests a moderate population density with potential for mixed traffic interactions. The absence of clear signage or speed control measures could also be a concern."
}
```
